In [37]:
# RLest_20231215ver
# Cross Entropyで推定したる！！
import pandas as pd
import os
import datetime
import numpy as np
from datetime import timedelta 
import csv
import time
import math
from scipy.stats import norm
from scipy.optimize import minimize
import random

In [38]:
df_data = pd.read_csv('/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/20230130_0845_20sec.csv')
search_folder = "/Users/takahiromatsunaga/res2023/bledata/ble_timegroup/20230130_08/split_files/group_3_1/group_2"
file_list = list(os.listdir(search_folder))

df_link = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_link_post.csv')
L = len(df_link)

df_node = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_stanw/stanw_node.csv')
df_ble = pd.read_csv('/Users/takahiromatsunaga/res2023/shibuya_nw/shibuya_nw_detail/ble_nw.csv')

In [46]:

#######################################
####### パラメタ&タイムステップ設定 #######
#######################################

xdim = 2
x0 = np.zeros(xdim)
x0[0] = -1
x0[-1] = 0.8

sigma1 = 2.0 # BLEの減衰定数．推定したいのと，混雑に影響されそう
sigma2 = 15 # 是非とも推定
T = 11 # 20ステップ以内ならT=21
#b = np.array([2, 15]) ## b[0]がBLE減衰定数，b[1]が正規分布の誤差分散

ddata = [66, 67, 68] # 吸収リンク linkidで指定
D = len(ddata)


In [47]:
#######################
######### 準備 ########
#######################

###### 各linkのoとdの座標を入れておく配列 ###### 
link_loc_array = [] 
for i in range(len(df_link)):
    linkid = df_link.loc[i, 'linkid']
    O = df_link.loc[i, 'o']
    D = df_link.loc[i, 'd']

    x_o = df_node[df_node['nodeid'] == O]['x'].iloc[0]
    y_o = df_node[df_node['nodeid'] == O]['y'].iloc[0]
    z_o = df_node[df_node['nodeid'] == O]['floor'].iloc[0] 
    o_loc = np.array([x_o, y_o, z_o])

    x_d = df_node[df_node['nodeid'] == D]['x'].iloc[0]
    y_d = df_node[df_node['nodeid'] == D]['y'].iloc[0]
    z_d = df_node[df_node['nodeid'] == D]['floor'].iloc[0]
    d_loc = np.array([x_d, y_d, z_d])

    loc_tuple = (o_loc, d_loc)
    link_loc_array.append(loc_tuple) 


###### 各ビーコンの座標を入れておく配列 ###### 
beacon_loc_array = []
for j in range(len(df_ble)):
    beacon_id = df_ble.loc[j, 'ID']
    x_j = df_ble[df_ble['ID'] == beacon_id]['x'].iloc[0]
    y_j = df_ble[df_ble['ID'] == beacon_id]['y'].iloc[0]
    z_j = df_ble[df_ble['ID'] == beacon_id]['floor'].iloc[0]
    
    beacon_loc = np.array([x_j, y_j, z_j])
    beacon_loc_array.append(beacon_loc) 


###### ビーコンとリンク線分の中点間距離を返す関数 ######
def dist_to_mid(p1, p2, x):
    p1 = np.array(p1)
    p2 = np.array(p2)
    x = np.array(x)

    segment = p2 - p1
    v1 = x - p1
    v2 = x - p2
    mid_point = (p1 + p2) / 2

    v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算
    distance = np.linalg.norm(x - mid_point)
    
    return distance


###### d_arrayの用意 ###### # 各リンクの中点とbleビーコンとの距離配列
d_array = np.zeros((len(df_link), len(df_ble)))
for i in range(len(df_link)):
    p_o = link_loc_array[i][0] # o座標 # i=1の時
    p_d = link_loc_array[i][1] # d座標

    for j in range(len(df_ble)):
        x_ap = df_ble.loc[j, 'x']
        y_ap = df_ble.loc[j, 'y']
        z_ap = df_ble.loc[j, 'floor']
        p_ap = np.array([x_ap, y_ap, z_ap])

        d_array[i, j] = dist_to_mid(p_o, p_d, p_ap) # ijはlink i+1, ビーコンj+1の距離


###### 混雑度dens，maxRSSI，meanRSSI，検出回数を各ビーコン各timestepごとに入れた配列（ビーコン数23*タイムステップ数） ######
# 15分間のデータでやるならtimstep18secなら50step, 20secなら45step
dens_jt = np.zeros((23, 50), dtype = int)
max_jt = np.zeros((23, 50), dtype = int)
mean_jt = np.zeros((23, 50), dtype = int)
#count_jt = np.zeros((23, 45), dtype = int)

grouped = df_data.groupby(['ID', 'timestep']) # listにして渡す必要がある
df_list = [group.reset_index(drop=True) for name, group in grouped]

for group in df_list:
    beacon_id = int(group.loc[0, 'ID'])
    timestep = int(group.loc[0, 'timestep'])

    # ユニークMACアドレス数，最大&平均RSSI
    unique_mac_count = group['MAC'].nunique()
    max_rssi = group['RSSI'].max()
    mean_rssi = group['RSSI'].mean()
    # countはそのタイムステップでの全検出数つまりgroupの長さそのもの
    #count = len(group)

    # numpy配列の要素はシーケンスダ
    dens_jt[beacon_id-1, timestep-1] = unique_mac_count
    max_jt[beacon_id-1, timestep-1] = max_rssi
    mean_jt[beacon_id-1, timestep-1] = mean_rssi
    #count_jt[beacon_id-1, timestep-1] = count


########################################
####### リンク接続行列＆プリズム準備 #######
########################################

def Imaking(link_data): 
    n = len(link_data)
    I = np.eye(n) # 全リンクに滞在リンク付与
    
    for i in range(n):

        if (i < 25) or (i >=  65): # 階段リンク以外の場合→これまで通り
            O = link_data.loc[i, 'o'] 
            D = link_data.loc[i, 'd'] 
            for j in range(n):
                if ((link_data.loc[j, 'o'] == O) or (link_data.loc[j, 'o'] == D)) or (link_data.loc[j, 'd'] == O) or (link_data.loc[j, 'd'] == D): 
                    I[i, j] = 1

        if 25 <= i < 65: ### iが25~64の場合→階段リンクの場合
            O = link_data.loc[i, 'o'] # onode
            D = link_data.loc[i, 'd'] # dnode

            for j in range(n):
                if link_data.loc[j, 'o'] == D: # DをOnodeとするリンクを1にしている
                    I[i, j] = 1
                
                if link_data.loc[j, 'o'] == D or link_data.loc[j, 'd'] == D: # odいずれかがDと一致するリンク[からの]遷移は0
                    I[j, i] = 0
                
                if link_data.loc[j, 'o'] == O or link_data.loc[j, 'd'] == O: # odいずれかがOと一致するリンクへの遷移は0
                    I[i, j] = 0

    return(I)

I = Imaking(df_link)

for i in range(26, 65, 2): # 階段には滞在リンク入れない
    I[i-1, i-1] = 0
    I[i, i] = 0

for i in range(25, 46): # 階段リンクに入ったら半分のところで折り返さない
    I[i, i+20] = 0
    I[i+20, i] = 0

for i in range(66, 69): # 吸収リンクからは吸収リンクにしか接続しない
    I[i-1, :] = 0 
    I[i-1, i-1] = 1 

# 起点Oからの到達可能性指示変数io
Itlist = np.zeros((T, L, L)) ## これは各状態に対して与えるので遷移回数T-1よりひとつ多くてT成分考える（timestep数がTなら状態数はT）
II = np.copy(I)
Itlist[0, :, :] = np.eye(L) 

for ts in range(1, T): 
    Itlist[ts, :, :] = II 
    II = np.dot(II, I)
    II = np.where(II > 0, 1, 0) 

# 終点Dからの到達可能性指示変数id ## これもIlist同様の考え方
Ittlist = np.zeros((T, L, L))
for ts in range(T): 
    Ittlist[ts, :, :] = np.transpose(Itlist[T - ts - 1, :, :]) 



/var/folders/1p/_62pxgbn5kg837b06pfnvkdw0000gn/T/ipykernel_2886/1926983877.py:49: RuntimeWarning: invalid value encountered in double_scalars
  v = v1 - np.dot(v1, segment) / np.dot(segment, segment) * segment # 垂直ベクトルを計算


In [48]:
#####################
#### od_list作成 #### # ビーコン列からODを特定（ちょっとよくないけどとりあえず）
#####################

saikyo_key = [1, 2, 3, 4, 5, 6]
yamate_key = [7, 8, 9, 10, 11, 12, 13]
discriminate_key = [19, 22, 23]
abs_key = [16, 20, 21]

od_list = pd.DataFrame(columns=['userid', 'o', 'd', 'abs', 'Ti'])

count_file = 0
for file_name in file_list:

    dfi = pd.read_csv(os.path.join(search_folder, file_name))   # file_path = os.path.join(search_folder, file_name)
    userid = int(file_name.split('.')[0])                       # userid = int(base_name)

    Ti = dfi['timestep'].nunique()                              # 個人の滞在時間 Tiまでにlast_linkにくる   
    grouped = dfi.groupby('timestep')
    dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

    ### 各人の各時刻の最大スコアのbeacon列 ###
    max_beacon_list = [] 
    for t in range(Ti):
        dfit = dfi_list[t] # 時刻tの全ログデータ
        ts = int(dfit.loc[0, 'timestep'])

        captured_beacon = dfit.groupby('ID')
        captured_beacon_list = [group.reset_index(drop=True) for name, group in captured_beacon]
        tdict = {}

        for j in range(len(dfit)):
            jid = dfit.loc[j, 'ID'] # そのログデータのビーコン
            max_rssi = max_jt[jid-1, ts-1] # そのビーコンの時刻tsでのmax_rssi
            rssi = dfit.loc[j, 'RSSI'] # そのログの電波強度
            jscore = max_rssi / rssi # このログのスコア→これを辞書のjidのところに足すイメージ
            if jid not in tdict:
                tdict[jid] = 0
            tdict[jid] += jscore  

        jmax = max(tdict, key = tdict.get) 
        max_beacon_list.append(jmax) #### なんかここ空になってる，なんで→append使う時はa = a.append()ってしないでa.append()だけにするらしいわ
        # これはGPTに言われるまで知らなかった，，，

    ### Oの確定 ### 
    first_beacon = max_beacon_list[0]
    first_dist_list = d_array[:, first_beacon-1] 
    first_dist_dict = {index + 1: value for index, value in enumerate(first_dist_list)} # keyはlinkidになっている．max_beaconと各リンクの距離が入ってる

    if any(item in discriminate_key for item in max_beacon_list): # 埼京オンリーにする
        filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in saikyo_key}
        max_key = min(filtered_dict, key = filtered_dict.get) # これが初期リンクとなる．埼京ホームのうち，最もmax_beaconに近いリンク
    else:
        filtered_dict = {key: first_dist_dict[key] for key in first_dist_dict if key in yamate_key}
        max_key = min(filtered_dict, key = filtered_dict.get)

    a0 = int(max_key)       # linkid
        
    #### Dの確定 #### 
    last_beacon = max_beacon_list[-1]

    if last_beacon not in abs_key: 
        continue
        
    last_link = 0
    if last_beacon == 16: # node39
        last_link = 25 
        abs_link = 46
    elif last_beacon == 20: # node28
        last_link = 17
        abs_link = 47
    elif last_beacon == 21: # node29
        last_link = 18   
        abs_link = 48   
    else:
        continue

    ## ここまで条件を満たせばod_listにデータを追加
    od_list.loc[count_file, 'userid'] = userid
    od_list.loc[count_file, 'o'] = a0
    od_list.loc[count_file, 'd'] = last_link
    od_list.loc[count_file, 'abs'] = abs_link
    od_list.loc[count_file, 'Ti'] = Ti

    count_file += 1

OD = len(od_list)


##########################
###### 時空間プリズム ######
##########################

Ilist = []
for i in range(OD):   

    # OD確定読み込み
    ao = od_list.loc[i, 'o'] - 1  # index
    ad = od_list.loc[i, 'd'] - 1  # index
    aabs = od_list.loc[i, 'abs'] - 1
    userid = od_list.loc[i, 'userid']

    Id = np.zeros((T-1, L, L)) # Id[t]は時刻tの時の利用可能遷移k→aを示す．最後のt=T-1のとき不要なので要素数はt=0~T-2のT-1個

    for ts in range(T-1): # Tまでに吸収されるのでdはaabs．遷移回数はT-1なのでこれでOK
        if ts == 0:
            Id[ts, ao, :] = I[ao, :] 
            continue
        
        alist = np.where(Itlist[ts + 1, ao, :] == Ittlist[ts + 1, aabs, :])[0] ## ts=1（つまり第二回目の状態）のとき3番目の状態を見ている→OK
        ## Itlistの長さはT．最後はT-2で，そのときT-2+1=T-1, 
        for a in alist:
            if Itlist[ts + 1, ao, a] == Ittlist[ts + 1, aabs, a]: # always True
                klist = np.where(I[:, a] == 1)[0]

                for k in klist:
                    if len(np.where(Id[ts - 1, :, k] == 1)[0]) != 0:
                        Id[ts, k, a] = 1 

    Ilist.append(Id)


In [49]:
#######################
###### 観測モデル #######
#######################

def qjx(rssi, max_rssi, x_loc, j, b):               # bは観測誤差分散のパラメータ．x_loc（x, y, z)にいたときビーコンjでrssiの観測が得られる確率
    j_loc = beacon_loc_array[j-1]                   # id=jのビーコンの位置座標
    #s = max_rssi
    d_est = 10 ** ((max_rssi - rssi)/(10*b[0]))     # 減衰パラメタb[0] rssi0→-40にしてみる
    d = np.linalg.norm(x_loc - j_loc) 
    f = norm.pdf(d, d_est, b[1])                    # 誤差分散b[1]
    omega = (100 + rssi) / 10 

    return omega * f


def Mmodel(Pall, b): # 観測尤度を返す # xの代わりにPallを入れる．とにかくこれが本命！！！肝！！！！！！
    print(f'bは{b}')
    count = 0 # 観測結果に入ったサンプル数（後で出力する）
    
    SLlist = [] 
    TLlist = []

    for od in range(OD):

        # odに対応するuserのファイルを読み込む（od_listのuserid列が実データとod_listを紐づけている）
        userid = od_list.loc[od, 'userid']
        file_name = f'{userid}.csv'
        dfi = pd.read_csv(os.path.join(search_folder, file_name)) 
        grouped = dfi.groupby('timestep')
        dfi_list = [group.reset_index(drop=True) for name, group in grouped]  

        Ti = od_list.loc[od, 'Ti']              # 個人odの観測限界．時刻Tiでlast_linkに到達し，時刻Ti+1で吸収される
        ao = od_list.loc[od, 'o'] - 1           # index
        ad = od_list.loc[od, 'd'] - 1           # index
        aabs = od_list.loc[od, 'abs'] - 1

        #### プリズム用意 #### odを与えたらプリズム制約．時刻Ti+1つまりTi回の遷移でaoからabsまで行く．もしくは時刻TiつまりTi-1回の遷移でaoからlastまで行く
        Idm = np.zeros((Ti, L, L)) ## 1成分目の値が遷移回数．Tiにするならabsまで行くことを仮定している．そのときItimlistは状態数＝時間数なのでTi+1で良い
        Itmlist = np.zeros((Ti+1, L, L))
        IIm = np.copy(I)
        Itmlist[0, :, :] = np.eye(L)

        for tt in range(1, Ti+1): 
            Itmlist[tt, :, :] = IIm 
            IIm = np.dot(IIm, I) 
            IIm = np.where(IIm > 0, 1, 0)

        Ittmlist = np.zeros((Ti+1, L, L))
        for tt in range(Ti+1): 
            Ittmlist[tt, :, :] = np.transpose(Itmlist[(Ti+1) - tt - 1, :, :])

        for tt in range(Ti):
            if tt == 0:
                Idm[tt, ao, :] = I[ao, :] 
                continue

            alist = np.where(Itmlist[tt + 1, ao, :] == Ittmlist[tt + 1, aabs, :])[0] # ここadじゃなくてaabsが適切では????
            for a in alist:
                if Itmlist[tt + 1, ao, a] == Ittmlist[tt + 1, aabs, a]: # ここadじゃなくてaabsが適切では????
                    klist = np.where(I[:, a] == 1)[0] 

                    for k in klist:
                        if len(np.where(Idm[tt - 1, :, k] == 1)[0]) != 0:
                            Idm[tt, k, a] = 1 ### 時刻tにおいて，a(t)→a(t+1)への遷移がOKなところが1になっている                            

        # SLmatrix = np.zeros((T, L)) # status_likelihood 状態数は遷移数+1 ### 今回不要
        TLmatrix = np.zeros((T-1, L, L)) # 条件付きの遷移尤度
        pmmatrix = np.zeros((T-1, L, L))  # p(m|a)のmatrixのはず
        pmstmatrix = np.zeros((T-1, L, L)) # 同時発生尤度のマトリックス
        # SLmatrix[0, ao] = 1 # t = 0ではaoに確定的に存在するとする ####### ←ここがまずよくない．

        klist = [ao] # 遷移元として有効なリンクのindex

        for t in range(1, T): #### t=tでは，k=a(t-1)からa=a(t)への遷移を考える．TLmatrixは遷移数＝T-1個しかないのでindexは0 ~ t-2

            #######################################
            ######### 1. 遷移発生尤度を計算 ##########
            ####################################### #### ここを同時分布ではなく条件付き分布にしないといけない，だいぶ面倒

            if t <= Ti - 1: # 観測の範囲内で考える，観測のないところは吸収に尤度1で滞在とする

                dfit = dfi_list[t] 
                ts = int(dfit.loc[0, 'timestep'])

                #slsum = 0
                #nlist = np.argsort(SLmatrix[t-1])[::-1]  # tsにおける状態尤度が大きい順に並び替えたインデックスリスト
                ## t=1のときSL[0]を参照するがaoしかないのでそのままaoを見ることになる．t=2のときSL[1]を参照，tのときt-1を参照
                ## ループは1からまでにしてt-1における尤度が大きいリンクに対して??実装?
                ## t-1で尤度大きいリンクをkとしてtのリンクをaとして扱った方が実態に近い

                #klist = [] # つまり時刻t-1で状態尤度が高い上位のリンクをklistに入れている

                #for i in range(L):
                #    slsum += SLmatrix[t-1, nlist[i]]
                #    klist.append(nlist[i])
                #    if slsum >= 0.9:
                #        break
                
                #klist3 = [i+1 for i in klist]
                
                #print(f'時刻{t-1}での状態発生尤度の高かったリンクは{klist3}') ##

                ########
                ##### klistを作る！！！！！！
                ########

                for k in klist: # range(L): ## t-1での全リンクに対して一応（あでもこれはt-1の時に(Q(st-1)!=0だったst-1をkの集合として扱うべきですね．)

                    for a in range(L): ## tのリンクはこっち

                        if Idm[t-1, k, a] == 0: ### t-1時点でk→aが利用可能か否かを見る ###### 
                            TLmatrix[t-1, k, a] = 0 ### 多分これでいいのか？観測的に利用が不可なら観測対象から外すということ．
                            continue
                            ## 利用可能なら
                        
                        ## このリンクa=a(t)の位置情報
                        a_loc = link_loc_array[a]
                        a_o = a_loc[0]
                        a_d = a_loc[1]

                        ## 位置については一旦保留→aの上に確率的に存在するという仮定が望ましいが一旦リンクの中点にいるとする
                        x_loc = (a_o + a_d) / 2

                        pm = 1

                        for j in range(len(dfit)): ### ここでようやく各観測ログにアプローチ
                            ### rssi小さい観測がpmの極小化を招いている説もある，積をとっているので．
                            ### でも相対評価で尤度最大化しようとしているならそこまで気にならないはずだが
                            j_id = dfit.loc[j, 'ID']
                            j_rssi = dfit.loc[j, 'RSSI']
                            j_max_rssi = max_jt[j_id - 1, ts - 1] 

                            pm *= qjx(j_rssi, j_max_rssi, x_loc, j_id, b) # 観測方程式
                        
                        pmmatrix[t-1, k, a] = pm # 時刻tの観測尤度はリンクaの観測確率ね→ただしindexの関係でt-1に入れる．つまりt=1にat-1=a0=k→at=a1=aの観測尤度がpm[0]に入ってる
                        #print(f'時刻{t}でリンクa(t)つまり{a+1}の観測尤度{pm}')
                
                # 遷移尤度のnume pmst 時刻tにおいてk(t-1)→a(t)への遷移が観測された確率がt-1に入ってる
                # 時刻tにおいてk(t-1)→a(t)への遷移確率がPall[t-1]に入っている=時刻t-1にk(t-1)→a(t)に遷移する確率がPall[t-1]ということ
                # つまり時刻tにおけるat→at+1への遷移確率はPall[t]に入っているということか．で，aT-1→aTへの遷移確率まで入っているがaT-2→aT-1までしか使わない
                pmstmatrix[t-1, :, :] = pmmatrix[t-1, :, :] * Pall[od, t-1, :, :] # 観測確率pm*遷移確率（正規化前の観測尤度）

                # 正規化
                deno = np.sum(pmstmatrix[t-1, :, :], axis=1) # np.sum(pmstmatrix[t-1], axis=1)になってた
                deno = (deno == 0) * 1 + (deno != 0) * deno ## 観測尤度の分母
                
                deno_rep = np.tile(deno, L).reshape(L, L).T ##### これがないと変になった

                pmstmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] / deno_rep ## これが観測尤度

                #sl = SLmatrix[t-1, :] # このままでは行ベクトル
                #sl_col = sl[:, None] #### ここがRと文法異なる部分

                TLmatrix[t-1, :, :] = pmstmatrix[t-1, :, :] # 状態を事前分布として与えないで条件付き分布を考えるからこれでいい* sl_col # 同時発生尤度を計算してる（SLmatrixはindexが1増えてるので，同じt-1でもひとつ前の段階を指名している
                # 時刻tにk(a_t-1)からa(a_t)への遷移が発生する尤度

                #### klist更新
                klist = []
                TLt = TLmatrix[t-1, :, :]
                # 方法1: any() を使って非零の要素を持つ列を見つける
                non_zero_columns = np.any(TLt != 0, axis=0)
                klist = np.where(non_zero_columns)[0]

                # 方法2: nonzero() を使って非零の要素がある列のインデックスを取得する
                non_zero_indices = np.nonzero(np.any(TLt != 0, axis=0))[0]
                klist = non_zero_indices.tolist()

                ## 列が非零のインデックスを次のklistに入れる


                # 非ゼロ要素のインデックスを取得
                #non_zero_indices = np.transpose(np.nonzero(TLmatrix[t-1, :, :])) +1

                # タプル形式のリストに変換
                #non_zero_tuples = [tuple(index) for index in non_zero_indices]

            elif t == Ti:
                TLmatrix[t-1, ad, aabs] = 1 ### それ以外はそのまま→0のまま
            
            else:
                TLmatrix[t-1, aabs, aabs] = 1 ### それ以外はそのまま→0のまま


            ########################################
            ########## 2. 状態発生尤度を定義
            ### 今回は同時分布を考えないのでこっちは考えなくていい！！！ ##########
            ########################################
            """
            if t <= Ti - 2: # 観測プリズム（ao, ad, Tiから決定）Idm内の状態しか取り得ない．k(a_t-1)からa(a_t)への遷移がOKのときIdmt[t-1]が1

                dfit = dfi_list[t] # t<len(dfit)=Tiにおいて指定可能
                ts = int(dfit.loc[0, 'timestep'])

                for i in range(L): ## iは遷移先リンク，つまりa(t)の候補ということになる

                    klist = np.where(Idm[t-1, :, i] == 1)[0] ## 前時刻参照．時刻tにおいてリンクi=a(t)に遷移可能な前リンクk=a(t-1)を探し，klistに入れている．この時，構造化NWと観測情報からリンク候補はかなり絞られている

                    if len(klist) == 0:
                        continue

                    kTL = 0
                    for k in klist: # このiに移動できる元リンク（t-1時点でのリンクa(t-1)）
                        kTL += TLmatrix[t-1, k, i] ## 時刻t-1にk=a(t-1)からi = a(t)への遷移発生尤度をkTLに足していき，これがtでのiの存在尤度になる（それはそう！！）
                        #print(f'状態遷移 k = a(t-1) = {k+1} → a = a(t) = {i+1}（全部リンクid）の遷移発生尤度は{kTL}')

                    SLmatrix[t, i] = kTL ### SLmatrixはindex t に入れる（SL[0]を事前に定義済みなので）

            if t == Ti - 1: # この時刻で最終リンクに確定的に滞在
                #print(f'時刻{t}で最後のリンク')
                SLmatrix[t, ad] = 1

            elif t >= Ti: # この時刻以降は確定的に吸収に滞在
                #print(f'時刻{t}で吸収リンク')             
                SLmatrix[t, aabs] = 1
            """
        # SLlist.append(SLmatrix)

        TLlist.append(TLmatrix)
    
    return(TLlist) 




In [50]:

###########################
####### 経路選択モデル ######
###########################

###### 即時効用行列 ###### # 混雑内生性考えないならT次元は圧縮可能だが一応→時不変だし，経路長も一緒だし経路パラメタしか入れてないので全リンクの効用が等しくなるよ．
def Mset(x): 
    #Probs_T = Probs.T # Probsを読み込む（内生性バージョン）
    inst = np.zeros((T-1, L, L))
    for t in range(T-1):
        inst_t = np.exp(df_link['length']/10 * x[0]) # + Probs_T[:, t] * x[1])    # + df_link['staire'] * x[1]) # + df_link_integrated['staire_with_esc'] * x[3])
        inst_t = pd.concat([inst_t]*L, axis=1) 
        inst_t = inst_t.T
        inst_t_numpy = inst_t.values # DataFrameをNumPy配列に変換
        inst[t, :, :] = inst_t_numpy

    return inst



###### 価値関数更新 ###### # 状態価値関数なので状態の数＝Tだけある
# 山野さんのコードは遷移回数がTなので状態数はT+1
# 今回のコードは状態数がTなので遷移回数はT-1
"""
def newV(x):
    beta = x[-1]
    z = np.ones((T, L, OD)) # なんでODが第三成分なのかがよくわからないが
    V = np.zeros((T, L, OD))

    for od in range(OD):
        # print(od_list.loc[od, 'userid'])
        Id = Ilist[od]
        #zI = np.ones((T, L))
        #VI = np.ze
        ##### 即時効用 #####
        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] # 内生性考慮してないのでMsetは定常
            M[ts, :, :] = Mts

        ##### 価値関数 ##### ### ここで価値関数求解（newVに相当）が入っている
        for t in range(T-1, 0, -1): # T-1から1までのT-1回

            ####
            zi = np.dot(M[t-1, :, :], (z[t, :, od] ** beta) )
            #zi = zii.sum(axis = 1)
            z[t-1, :, od] = (zi==0)*1 + (zi!=0)*zi
            V[t-1, :, od] = np.log(z[t-1, :, od])

    return z
"""

###### リンク選択確率計算 ###### # 山野さんコードにおけるnewVを吸収している
### が，厳密には意味が異なる気がする．newV分離させて構造推定した方が良さそう
## いややっぱ合体させる
def newPall(x):
    #print(f'経路選択パラメタ{x}と観測パラメタ{b}で観測モデル回す')
    # Probs = np.zeros((T, L)) ## 時間tでのリンクkの期待人数＝個人ごとの選択確率の総和
    Pall = np.zeros((OD, T-1, L, L)) # 個人ごと，時刻ごとの各リンク間遷移確率行列 遷移の回数なのでT-1．今回Tを状態の数としているので遷移数はT-1になる
    beta = x[-1]
    

    #### 以降個人ごとに処理 #### 
    for od in range(OD): 
        
        Id = Ilist[od] # すでに個人のプリズムはIlistで用意してある

        ##### 即時効用 #####
        M = np.zeros((T-1, L, L))
        for ts in range(T-1):
            Mts = Id[ts, :, :] * Mset(x)[ts, :, :] # 内生性考慮してないのでMsetは定常
            M[ts, :, :] = Mts

        ##### 価値関数 ##### ### ここで価値関数求解（newVに相当）が入っている
            
        z = np.ones((T, L))
        for t in range(T-1, 0, -1):

                ####
            zii = M[t-1, :, :] * (z[t, :] ** beta) 
            zi = zii.sum(axis = 1)
            z[t-1, :] = (zi==0)*1 + (zi!=0)*zi
            
            """
            ### この時刻のV
            z = np.exp(beta * V[ts+1, :, od]).reshape(L, 1)
            ZD = np.tile(z, L).reshape(L, L).T
            
            Mz = (M @ z != 0) * (M @ z) + (M @ z == 0)
            MZ = np.tile(Mz, L).reshape(L, L)

            p = (M * ZD) / MZ
            """

            
        ##### 選択確率行列 ##### # 尤度計算は不要
        for t in range(T-1): # 選択確率は各時刻tごとに決まる
            for k in range(L):
                for a in range(L):
                    if M[t, k, a] == 0: # 接続条件を満たせなかった観測は排除（logzero回避）
                        continue # ここがbreakになってたのが癌だった
            
            
                    Pall[od, t, k, a] += np.exp(np.log(M[t, k, a]) + beta * np.log(z[t+1, a]) - np.log(z[t, k])) 

    return Pall # 個人ごとの各時刻ごとのリンク遷移確率の一覧がPallに入った



In [51]:
def KL(x): 
    KL = 0
    LL = 0
    Pt = newPall(x)

    TLlist = Mmodel(Pt, b)
    for od in range(OD):
        userid = int(od_list.loc[od, 'userid'])
        TLmatrix = TLlist[od]
        for t in range(T-1):
            KLit = 0
            pit = Pt[od, t, :, :] # 時刻tにk(a_t-1)からa(a_t)への遷移確率はpall[t-1]に入っている
            pit = (pit == 0) * 1 + (pit != 0) * pit

            for k in range(L): # 時刻tにa(t-1)からa(tへの遷移を考える)
                for a in range(L):
                    logpka = 0
                    if pit[k, a] == 0:
                        logpka = 0
                        continue
                    
                    if TLmatrix[t, k, a] == 0:
                        continue

                    else:
                        p = pit[k, a]
                        
                        q = TLmatrix[t, k, a]
                        print(f'時刻{t}に個人{userid}が{k+1}から{a+1}へ遷移する確率{p}と尤度{q}')
                    qlp = q * np.log(p)
                    qlq = q * np.log(q)
                    print(qlp)

                    # KLit += (qlq - qlp)
                    KLit += qlp
            
            KL += KLit
            #print(f'user{userid}，時刻{t}の一般化平均情報量{-KLit}（正，小さい方がいい）')

    print(f'パラメタ{x}での一般化平均情報量{-KL}')

    return -KL



In [30]:
x1 = np.array([-1.5,  0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 2 15]
時刻0に個人1006が7から7へ遷移する確率0.36560496268383963と尤度0.028448775495748795
-0.028625210972559278
時刻0に個人1006が7から8へ遷移する確率0.5916118070879333と尤度0.9382487781908242
-0.4924910906604969
時刻0に個人1006が7から34へ遷移する確率0.04278323022822723と尤度0.03330244631342718
-0.10495629186365196
時刻1に個人1006が7から7へ遷移する確率0.3700481254258108と尤度0.001261797663744451
-0.0012543810859454276
時刻1に個人1006が7から8へ遷移する確率0.6299518745741887と尤度0.9987382023362555
-0.4615287604238279
時刻1に個人1006が8から7へ遷移する確率0.20275765987822905と尤度0.00034336708357514634
-0.000547925897068172
時刻1に個人1006が8から8へ遷移する確率0.3451647479029688と尤度0.27178194542983247
-0.28910354531245547
時刻1に個人1006が8から9へ遷移する確率0.3451647479029688と尤度0.5546353820421254
-0.5899842060903356
時刻1に個人1006が8から30へ遷移する確率0.10691284431583384と尤度0.17323930544446703
-0.38731827265796315
時刻2に個人1006が7から8へ遷移する確率0.6561745926494553と尤度1.0
-0.42132837811747936
時刻2に個人1006が8から8へ遷移する確率0.33744885213865583と尤度0.011200071311604495
-0.012167100357721871
時刻2に個人1006が8から9へ遷移する確率0.33744885213865583と尤度0.6872617456616719
-0.7466

In [31]:
x1 = np.array([0, 0]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 2 15]
時刻0に個人1006が7から7へ遷移する確率0.3333333333333333と尤度0.031862683833216845
-0.03500473600911874
時刻0に個人1006が7から8へ遷移する確率0.3333333333333333と尤度0.6493996242278322
-0.7134384074331491
時刻0に個人1006が7から34へ遷移する確率0.3333333333333333と尤度0.31873769193895096
-0.35016914522584186
時刻1に個人1006が7から7へ遷移する確率0.5と尤度0.002146120835156814
-0.0014875776060299007
時刻1に個人1006が7から8へ遷移する確率0.5と尤度0.9978538791648432
-0.6916596029539154
時刻1に個人1006が8から7へ遷移する確率0.25と尤度0.00042164869309511846
-0.000584529205611334
時刻1に個人1006が8から8へ遷移する確率0.25と尤度0.19604850628972495
-0.2717809387754231
時刻1に個人1006が8から9へ遷移する確率0.25と尤度0.4000833757107037
-0.5546333277255591
時刻1に個人1006が8から30へ遷移する確率0.25と尤度0.4034464693064763
-0.5592955654132972
時刻2に個人1006が7から8へ遷移する確率0.5と尤度1.0
-0.6931471805599453
時刻2に個人1006が8から8へ遷移する確率0.25と尤度0.008088628156670847
-0.01121321960278837
時刻2に個人1006が8から9へ遷移する確率0.25と尤度0.49633654575056346
-0.6880685545917307
時刻2に個人1006が8から30へ遷移する確率0.25と尤度0.4955748260927657
-0.6870125869253715
時刻2に個人1006が9から8へ遷移する確率0.25と尤度0.008088628156670847
-0.01121

In [32]:
x1 = np.array([1, 0]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 2 15]
時刻0に個人1006が7から7へ遷移する確率0.33333333333333337と尤度0.03186268383321684
-0.03500473600911873
時刻0に個人1006が7から8へ遷移する確率0.33333333333333337と尤度0.6493996242278322
-0.713438407433149
時刻0に個人1006が7から34へ遷移する確率0.33333333333333337と尤度0.31873769193895096
-0.35016914522584175
時刻1に個人1006が7から7へ遷移する確率0.49999999999999994と尤度0.002146120835156814
-0.0014875776060299009
時刻1に個人1006が7から8へ遷移する確率0.49999999999999994と尤度0.9978538791648432
-0.6916596029539155
時刻1に個人1006が8から7へ遷移する確率0.24999999999999994と尤度0.00042164869309511846
-0.0005845292056113341
時刻1に個人1006が8から8へ遷移する確率0.24999999999999994と尤度0.19604850628972495
-0.27178093877542314
時刻1に個人1006が8から9へ遷移する確率0.24999999999999994と尤度0.4000833757107037
-0.5546333277255592
時刻1に個人1006が8から30へ遷移する確率0.24999999999999994と尤度0.4034464693064763
-0.5592955654132972
時刻2に個人1006が7から8へ遷移する確率0.49999999999999994と尤度1.0
-0.6931471805599454
時刻2に個人1006が8から8へ遷移する確率0.24999999999999994と尤度0.008088628156670847
-0.011213219602788371
時刻2に個人1006が8から9へ遷移する確率0.24999999999999994と尤度0.49633654575056346
-0.68

In [33]:
x1 = np.array([-2, 0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 2 15]
時刻0に個人1006が7から7へ遷移する確率0.36560496268383896と尤度0.02844877549574874
-0.02862521097255927
時刻0に個人1006が7から8へ遷移する確率0.5916118070879333と尤度0.938248778190824
-0.49249109066049673
時刻0に個人1006が7から34へ遷移する確率0.042783230228227304と尤度0.033302446313427235
-0.10495629186365207
時刻1に個人1006が7から7へ遷移する確率0.37004812542581145と尤度0.0012617976637444512
-0.0012543810859454254
時刻1に個人1006が7から8へ遷移する確率0.6299518745741898と尤度0.9987382023362555
-0.4615287604238261
時刻1に個人1006が8から7へ遷移する確率0.2027576598782294と尤度0.00034336708357514644
-0.0005479258970681716
時刻1に個人1006が8から8へ遷移する確率0.3451647479029694と尤度0.2717819454298325
-0.289103545312455
時刻1に個人1006が8から9へ遷移する確率0.3451647479029694と尤度0.5546353820421256
-0.5899842060903349
時刻1に個人1006が8から30へ遷移する確率0.10691284431583384と尤度0.17323930544446678
-0.3873182726579626
時刻2に個人1006が7から8へ遷移する確率0.656174592649453と尤度1.0
-0.4213283781174829
時刻2に個人1006が8から8へ遷移する確率0.3374488521386552と尤度0.011200071311604491
-0.012167100357721887
時刻2に個人1006が8から9へ遷移する確率0.3374488521386552と尤度0.6872617456616715
-0.7466008384

In [34]:
x1 = np.array([-5, 0.9]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 2 15]
時刻0に個人1006が7から7へ遷移する確率0.35593244195974805と尤度0.02678669848158155
-0.02767104354329713
時刻0に個人1006が7から8へ遷移する確率0.6252634505957547と尤度0.9590568173262196
-0.45035600743099896
時刻0に個人1006が7から34へ遷移する確率0.01880410744448937と尤度0.014156484192198896
-0.05625333742421161
時刻1に個人1006が7から7へ遷移する確率0.3464263481400699と尤度0.0011386986047941155
-0.0012071173587741262
時刻1に個人1006が7から8へ遷移する確率0.6535736518599345と尤度0.9988613013952058
-0.4248157600087715
時刻1に個人1006が8から7へ遷移する確率0.1852367111221581と尤度0.0003061293176618427
-0.000516170995370182
時刻1に個人1006が8から8へ遷移する確率0.3494706288843893と尤度0.2685352624017853
-0.2823207237453028
時刻1に個人1006が8から9へ遷移する確率0.3494706288843893と尤度0.5480097569337955
-0.5761422533981817
時刻1に個人1006が8から30へ遷移する確率0.11582203110906232と尤度0.1831488513467573
-0.3948140668631589
時刻2に個人1006が7から8へ遷移する確率0.6859786136394033と尤度1.0
-0.37690882719498126
時刻2に個人1006が8から8へ遷移する確率0.33884060055892473と尤度0.010799584173620163
-0.011687585241948207
時刻2に個人1006が8から9へ遷移する確率0.33884060055892473と尤度0.6626869477065034
-0.717176704

In [35]:
x1 = np.array([-10, 0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 2 15]
時刻0に個人1006が7から7へ遷移する確率0.3656049626838351と尤度0.028448775495748552
-0.028625210972559385
時刻0に個人1006が7から8へ遷移する確率0.5916118070879312と尤度0.9382487781908243
-0.4924910906605003
時刻0に個人1006が7から34へ遷移する確率0.04278323022822715と尤度0.03330244631342725
-0.10495629186365224
時刻1に個人1006が7から7へ遷移する確率0.3700481254258127と尤度0.0012617976637444466
-0.0012543810859454167
時刻1に個人1006が7から8へ遷移する確率0.6299518745741942と尤度0.9987382023362555
-0.4615287604238191
時刻1に個人1006が8から7へ遷移する確率0.2027576598782287と尤度0.00034336708357514525
-0.0005479258970681709
時刻1に個人1006が8から8へ遷移する確率0.3451647479029694と尤度0.2717819454298325
-0.289103545312455
時刻1に個人1006が8から9へ遷移する確率0.3451647479029694と尤度0.5546353820421256
-0.5899842060903349
時刻1に個人1006が8から30へ遷移する確率0.10691284431583384と尤度0.17323930544446678
-0.3873182726579626
時刻2に個人1006が7から8へ遷移する確率0.6561745926494553と尤度1.0
-0.42132837811747936
時刻2に個人1006が8から8へ遷移する確率0.33744885213865405と尤度0.01120007131160449
-0.012167100357721925
時刻2に個人1006が8から9へ遷移する確率0.33744885213865405と尤度0.6872617456616715
-0.746600838

In [36]:
x1 = np.array([-7, 0.8]) 
b1 = np.array([2, 15]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 2 15]
時刻0に個人1006が7から7へ遷移する確率0.36560496268384024と尤度0.02844877549574855
-0.02862521097255898
時刻0に個人1006が7から8へ遷移する確率0.5916118070879396と尤度0.9382487781908244
-0.492491090660487
時刻0に個人1006が7から34へ遷移する確率0.04278323022822776と尤度0.03330244631342725
-0.10495629186365175
時刻1に個人1006が7から7へ遷移する確率0.3700481254258075と尤度0.0012617976637444468
-0.0012543810859454347
時刻1に個人1006が7から8へ遷移する確率0.6299518745741852と尤度0.9987382023362555
-0.4615287604238333
時刻1に個人1006が8から7へ遷移する確率0.2027576598782258と尤度0.0003433670835751444
-0.0005479258970681744
時刻1に個人1006が8から8へ遷移する確率0.3451647479029645と尤度0.27178194542983186
-0.2891035453124582
時刻1に個人1006が8から9へ遷移する確率0.3451647479029645と尤度0.5546353820421241
-0.5899842060903412
時刻1に個人1006が8から30へ遷移する確率0.10691284431583384と尤度0.17323930544446883
-0.3873182726579672
時刻2に個人1006が7から8へ遷移する確率0.6561745926494645と尤度1.0
-0.4213283781174654
時刻2に個人1006が8から8へ遷移する確率0.33744885213865883と尤度0.011200071311604538
-0.012167100357721818
時刻2に個人1006が8から9へ遷移する確率0.33744885213865883と尤度0.6872617456616744
-0.7466008384

In [54]:
x1 = np.array([-5, 0.9]) 
b = np.array([4, 5])  # ここb1だったからううまくいかなかったということのはず．
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[4 5]
時刻0に個人1006が7から7へ遷移する確率0.35593244195974805と尤度1.896467849155647e-15
-1.9590784757790196e-15
時刻0に個人1006が7から8へ遷移する確率0.6252634505957547と尤度0.9999943687224853
-0.46957955275985974
時刻0に個人1006が7から34へ遷移する確率0.01880410744448937と尤度5.631277512796429e-06
-2.2376894556296477e-05
時刻1に個人1006が7から7へ遷移する確率0.3464263481400699と尤度4.9395826581360454e-30
-5.23637769171918e-30
時刻1に個人1006が7から8へ遷移する確率0.6535736518599345と尤度1.0
-0.4253000485807093
時刻1に個人1006が8から7へ遷移する確率0.1852367111221581と尤度2.7232702760321194e-36
-4.5917625263002984e-36
時刻1に個人1006が8から8へ遷移する確率0.3494706288843893と尤度5.513158630004072e-07
-5.796180809269627e-07
時刻1に個人1006が8から9へ遷移する確率0.3494706288843893と尤度0.7563890233328773
-0.7952188274656957
時刻1に個人1006が8から30へ遷移する確率0.11582203110906232と尤度0.2436104253512597
-0.5251511110003898
時刻2に個人1006が7から8へ遷移する確率0.6859786136394033と尤度1.0
-0.37690882719498126
時刻2に個人1006が8から8へ遷移する確率0.33884060055892473と尤度7.301640963688876e-21
-7.902021947999501e-21
時刻2に個人1006が8から9へ遷移する確率0.33884060055892473と尤度0.698644690167014
-0.7560910

こちらの尤度比（？？）は0.12程度．まあ，それっぽい観測誤差分散を与えているからか．観測誤差分散を変化させるとまた変わってくるのか．

x1 = np.array([-5, 0.9]) 
b = np.array([4, 10]) 
np.set_printoptions(threshold=np.inf)
print(KL(x1))

In [55]:
x1 = np.array([-5, 0.9]) 
b = np.array([4, 20])  # ここb1だったからううまくいかなかったということのはず．
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 4 20]
時刻0に個人1006が7から7へ遷移する確率0.35593244195974805と尤度0.06511148504081296
-0.06726109747985255
時刻0に個人1006が7から8へ遷移する確率0.6252634505957547と尤度0.9187299135174231
-0.4314192113379204
時刻0に個人1006が7から34へ遷移する確率0.01880410744448937と尤度0.016158601441763936
-0.0642091106002012
時刻1に個人1006が7から7へ遷移する確率0.3464263481400699と尤度0.00806096855095352
-0.008545312188335962
時刻1に個人1006が7から8へ遷移する確率0.6535736518599345と尤度0.9919390314490465
-0.4218717182643812
時刻1に個人1006が8から7へ遷移する確率0.1852367111221581と尤度0.0019225080609874055
-0.003241580737926029
時刻1に個人1006が8から8へ遷移する確率0.3494706288843893と尤度0.23657340577805056
-0.24871808097299028
時刻1に個人1006が8から9へ遷移する確率0.3494706288843893と尤度0.5722027856149013
-0.6015772495538123
時刻1に個人1006が8から30へ遷移する確率0.11582203110906232と尤度0.18930130054606056
-0.4080769045587496
時刻2に個人1006が7から8へ遷移する確率0.6859786136394033と尤度1.0
-0.37690882719498126
時刻2に個人1006が8から8へ遷移する確率0.33884060055892473と尤度0.036480882679484676
-0.03948054102485532
時刻2に個人1006が8から9へ遷移する確率0.33884060055892473と尤度0.6469453279609364
-0.700140722630

In [56]:
x1 = np.array([-5, 0.9]) 
b = np.array([5, 15])  # ここb1だったからううまくいかなかったということのはず．
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 5 15]
時刻0に個人1006が7から7へ遷移する確率0.35593244195974805と尤度0.012782358600738442
-0.013204359681364833
時刻0に個人1006が7から8へ遷移する確率0.6252634505957547と尤度0.9761732805278877
-0.4583935938279484
時刻0に個人1006が7から34へ遷移する確率0.01880410744448937と尤度0.011044360871373829
-0.04388675537634668
時刻1に個人1006が7から7へ遷移する確率0.3464263481400699と尤度0.00027688672610648197
-0.00029352347679191494
時刻1に個人1006が7から8へ遷移する確率0.6535736518599345と尤度0.9997231132738935
-0.42518228864264485
時刻1に個人1006が8から7へ遷移する確率0.1852367111221581と尤度3.474579296766879e-05
-5.858560257485374e-05
時刻1に個人1006が8から8へ遷移する確率0.3494706288843893と尤度0.12545264558998598
-0.13189285229041986
時刻1に個人1006が8から9へ遷移する確率0.3494706288843893と尤度0.6575358697475417
-0.6912909722741462
時刻1に個人1006が8から30へ遷移する確率0.11582203110906232と尤度0.21697673886950453
-0.46773686025244876
時刻2に個人1006が7から8へ遷移する確率0.6859786136394033と尤度1.0
-0.37690882719498126
時刻2に個人1006が8から8へ遷移する確率0.33884060055892473と尤度0.0035618995528522083
-0.0038547784783145465
時刻2に個人1006が8から9へ遷移する確率0.33884060055892473と尤度0.670858021190257
-0

In [57]:
x1 = np.array([-5, 0.9]) 
b = np.array([3, 15])  # ここb1だったからううまくいかなかったということのはず．
np.set_printoptions(threshold=np.inf)
print(KL(x1))

bは[ 3 15]
時刻0に個人1006が7から7へ遷移する確率0.35593244195974805と尤度0.015878559215877053
-0.0164027793036708
時刻0に個人1006が7から8へ遷移する確率0.6252634505957547と尤度0.9722344602754618
-0.45654399395980266
時刻0に個人1006が7から34へ遷移する確率0.01880410744448937と尤度0.011886980508661054
-0.04723505613612894
時刻1に個人1006が7から7へ遷移する確率0.3464263481400699と尤度0.00042030459499122256
-0.00044555861441330835
時刻1に個人1006が7から8へ遷移する確率0.6535736518599345と尤度0.9995796954050088
-0.42512129301604085
時刻1に個人1006が8から7へ遷移する確率0.1852367111221581と尤度6.692849536572578e-05
-0.00011284952495048574
時刻1に個人1006が8から8へ遷移する確率0.3494706288843893と尤度0.15917114827874967
-0.16734231988569095
時刻1に個人1006が8から9へ遷移する確率0.3494706288843893と尤度0.6315700645059732
-0.6639921927289073
時刻1に個人1006が8から30へ遷移する確率0.11582203110906232と尤度0.2091918587199114
-0.4509549903728288
時刻2に個人1006が7から8へ遷移する確率0.6859786136394033と尤度1.0
-0.37690882719498126
時刻2に個人1006が8から8へ遷移する確率0.33884060055892473と尤度0.005201190622424873
-0.005628861054456226
時刻2に個人1006が8から9へ遷移する確率0.33884060055892473と尤度0.6686266865394295
-0.72